In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import cv2

import numpy as np
import torch
import json
import easyocr

import time
import os
import gc
import math
import re

In [ ]:
path = "."
root_dir = f"{path}/data/design/"
resources = {}

for apartment in os.listdir(root_dir):
    apt_path = os.path.join(root_dir, apartment)
    if not os.path.isdir(apt_path):
        continue

    resources[apartment] = {}
    for floor in os.listdir(apt_path):
        floor_path = os.path.join(apt_path, floor)
        if not os.path.isdir(floor_path):
            continue

        res = {}
        mapping_file = os.path.join(floor_path, "mapping.json")
        if os.path.exists(mapping_file):
            with open(mapping_file, "r", encoding="utf-8") as f:
                res["mapping"] = json.load(f)

        metadata_file = os.path.join(floor_path, "metadata.json")
        if os.path.exists(metadata_file):
            with open(metadata_file, "r", encoding="utf-8") as f:
                res["metadata"] = json.load(f)

        map_file = os.path.join(floor_path, "design.jpg")
        if os.path.exists(map_file):
            res["image"] = cv2.cvtColor(cv2.imread(map_file), cv2.COLOR_BGR2RGB) 

        resources[apartment][floor] = res

In [ ]:
def crop_from_original(resources, code, padding=0):
    tiles_list = resources["mapping"][code]
    offsets = resources["metadata"]
    min_x, min_y = float("inf"), float("inf")
    max_x, max_y = 0, 0

    for tile_name in tiles_list:
        off = offsets[tile_name]
        x, y, w, h = off["x"], off["y"], off["width"], off["height"]

        min_x = min(min_x, x) + padding
        min_y = min(min_y, y) + padding
        max_x = max(max_x, x + w) + padding
        max_y = max(max_y, y + h) + padding

    cropped = resources["image"][min_y:max_y, min_x:max_x]
    return cropped

In [ ]:
building_code="S6.06"
floor_number=20
floor_ranges = resources[building_code.replace(".", "")].keys()
for i, fr in enumerate(floor_ranges):
    try:
        _, start, end = fr.split("_")
        start, end = int(start), int(end)
        if start <= floor_number <= end:
            floor = fr
            break
    except ValueError:
            continue
        
floor_resources = resources[building_code.replace(".", "")][floor]

In [ ]:
index = 30
design_cropped = crop_from_original(floor_resources, f'CH{index:02d}')

h, w = design_cropped.shape[:2]
print(design_cropped.shape)
min_y = 1500
max_y = h - 340
min_x = 1440
max_x = w - 525

img_resized = design_cropped[min_y:max_y, min_x:max_x]
print(img_resized.shape)
plt.figure(figsize=(6, 6))
plt.imshow(img_resized)
plt.axis("off")
plt.show()

In [ ]:
index = 30
cv2.imwrite(
    f"data/image/CH{index:02d}.jpg", 
    cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR), 
    [cv2.IMWRITE_JPEG_QUALITY, 100]
)